In [11]:
'''
@Authors: Sean Tonthat

@Description: Helper functions to help clean the code

@Date: 6/8/22
'''

#Import dependencies
import cv2
import os
import openpyxl
import numpy as np
from shutil import copyfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy.linalg as LA

In [4]:
def trim_to_128_by_128_celebA(image):
    '''
    Crops input image and resizes to 128x128
    
    INPUT: 
        image : (np.array) image from CelebA dataset. Shape is (178x218x3)
    
    OUTPUT: 
        crop_image : (np.array) resized and cropped version of the image with shape (128x128x3)
    '''
    
    crop_img = image.copy() #image is 178x218
    crop_img = crop_img[20:198, 0:178] #Because height of image is too long we crop
    crop_img = cv2.resize(crop_img, (128, 128),interpolation =cv2.INTER_AREA) #resize to 128x128
    
    return crop_img






def trim_to_128_by_128_bitmoji(image):
    '''
    Crops input image and resizes to 128x128
    
    INPUT: 
        image : (np.array) image from Bitmoji dataset. Shape is (384x384x3)
    
    OUTPUT: 
        crop_image : (np.array) resized and cropped version of the image with shape (128x128x3)
    '''
    
    crop_img = cv2.resize(image, (128, 128),interpolation =cv2.INTER_AREA)
    return crop_img






def is_facing_forward(image, coord):
    #Takes the uncropped image and outputs a prediction for head pose
    '''
    INPUT: 
        image : (np.array) image from CelebA dataset. Shape is (218,178,3)
        coord : (list) list of 10 coordinates
        eyes have indexes 0-3, nose 4-5, mouth 6-9
    
    OUTPUT: boolean either FALSE for off-center or TRUE for center
    
    NOTE: images with prediction left, right will be removed from dataset
    
    '''
    size = image.shape
 
    #This array represents the coordinates in 2D space
    points_2D = np.array([
                        (coord[4], coord[5]),  # Nose tip
                        (coord[0], coord[1]),  # Left eye corner
                        (coord[2], coord[3]),  # Right eye corner
                        (coord[6], coord[7]),  # Left mouth 
                        (coord[8], coord[9]),   # Right mouth
                        (round((coord[8] + coord[6]) / 2), coord[7] - 20) #chin
                      ], dtype="double")
 

    #This array represents the coordinates in 3D space
    #This collection of coordinates are commonly used when information regarding the camera are not provided
    points_3D = np.array([
                      (0.0, 0.0, 0.0),         #Nose tip
                      (-225.0, 170.0, -135.0), #Left eye corner
                      (225.0, 170.0, -135.0),  #Right eye corner 
                      (-150.0, -150.0, -125.0),#Left mouth 
                      (150.0, -150.0, -125.0),  #Right mouth 
                      (0.0, -330.0, -65.0) #chin
                     ])
    
    
    #Get the coordinates in 3d space for the nose and project a point (0.0, 0.0, 1000.0) into 3D space
    dim = size[1]
    midpoint = (size[1] * 0.5, size[0] * 0.5)
    camera_matrix = np.array([[dim, 0, center[0]],[0, focal_length, midpoint[1]],[0, 0, 1]], dtype = "double")
    
    _, rotation_vector, translation_vector = cv2.solvePnP(points_3D, points_2D, camera_matrix, np.zeros((4,1)) , flags=0)
    nose_end_point2D, _ = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)
    
    
    #Get the angle between the projected point and the nose matrix
    point1 = ( int(points_2D[0][0]), int(points_2D[0][1]))
    point2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))

    inner = np.inner(point1, point2)
    norms = LA.norm(point1) * LA.norm(point2)

    cos = inner / norms
    rad = np.arccos(np.clip(cos, -1.0, 1.0))
    head_pose_angle = np.rad2deg(rad)
    
    
    #If false is returned then the image has a non-forward facing image
    if (abs(head_pose_angle) > 1.2):
        return False
    

    return True
    

# Cleaning Dataset 

In [3]:
#CSV contains landmark coordinates of mouth, eyes, nose for celebA dataset
dir_att = "C:\\Users\\Sean\Desktop\\UCSD Courses\\ECE228 Project\\img_align_celeba_unaltered\\list_landmarks_align_celeba.txt"
data_att = pd.read_csv(dir_att, delim_whitespace=True)
df_list = data_att.values.tolist() #collect csv file onto list for iterating


## Stage 1: Remove non-forward facing images from dataset

In [22]:
'''
This cell does the following:

1) reads image in a directory (presumably the celebA dataset images)
2) checks if images have subjects facing forward using is_facing_forward()
3) if image is facing foward then add it to a new directory
4) append the coordinates of the image onto an excel file
'''

# assign directory
src_img_dir = "C:\\Users\\Sean\\Desktop\\UCSD Courses\\ECE228 Project\\img_align_celeba_unaltered\\img_align_celeba"
tgt_img_dir = "C:\\Users\\Sean\Desktop\\UCSD Courses\\ECE228 Project\\img_align_celeba_ALTERED\\img_celebA\\"


# create an excel worksheet to store the landmarks of the front only dataset, it omits non-frontal images
wb_path = "C:\\Users\\Sean\\Desktop\\UCSD Courses\\ECE228 Project\\img_align_celeba_ALTERED\\celebA_landmarks_front_only.xlsx"
wb = openpyxl.load_workbook(filename = wb_path)
sheet = wb["DATA"] #set active sheet to a pre-existing sheet named "DATA" in the excel workbook


print("STARTING")


for i, filename in enumerate(os.scandir(src_img_dir)):
    
    if filename.is_file():
        coord = df_list[i][1:]
        image = mpimg.imread(filename.path)
        
        if is_facing_forward(image, coord): #image is facing forward, else ignore
            
            #WRITE landmarks/ ATTRIBUTES to excel file
            sheet.append(df_list[i]) #append landmarks
            
            #COPY file to target directory
            tgt = tgt_img_dir + df_list[i][0]
            copyfile(filename.path, tgt)
            

wb.save(wb_path)
wb.close()   


print("DONE")

STARTING
DONE


## Stage 2: Resize the Images

In [4]:
'''
Read excel file in earlier stage 1
Convert it to a list. The list will contain the coordinates of the forward facing images
'''

crop_tgt_dir =  "C:\\Users\\Sean\\Desktop\\UCSD Courses\\ECE228 Project\\img_aligned_cropped\\"
tgt_img_dir = "C:\\Users\\Sean\Desktop\\UCSD Courses\\ECE228 Project\\img_align_celeba_ALTERED\\img_celebA"

#CSV contains landmark coordinates of mouth, eyes, nose for celebA dataset
dir_att = "C:\\Users\\Sean\Desktop\\UCSD Courses\\ECE228 Project\\img_align_celeba_ALTERED\\celebA_landmarks_front_only.xlsx"
data_att = pd.read_excel(dir_att)
df_list = data_att.values.tolist()

print("DONE HERE")

DONE HERE


In [41]:
'''
read all the files in tgt_img_dir
crop and resize each image to 128x128 using trim_to_128_by_128()
'''

print("STARTING")
print(df_list[0])
for i, filename in enumerate(os.scandir(tgt_img_dir)):
    if filename.is_file():
        image = cv2.imread(filename.path)
        crop_img = trim_to_128_by_128(image) #crop image
        filename = crop_tgt_dir + df_list[i][0]
        cv2.imwrite(filename, crop_img) #save image to directory
        

        
print("DONE")

STARTING
['000007.jpg', 70, 112, 108, 111, 85, 135, 72, 152, 104, 152]
DONE


## Stage 3: Resize the Bitmoji Images to 128x128

In [14]:
orig_bitmoji_dir =  "C:\\Users\\Sean\Desktop\\UCSD Courses\\ECE228 Project\\bitmoji\\BitmojiDataset\\images"
tgt_bitmoji_dir ="C:\\Users\\Sean\\Desktop\\UCSD Courses\\ECE228 Project\\bitmoji\\BitmojiDataset\\images_resized\\"

#CSV contains landmark coordinates of mouth, eyes, nose for celebA dataset
dir_att = "C:\\Users\\Sean\Desktop\\UCSD Courses\\ECE228 Project\\bitmoji\\BitmojiDataset\\landmarks.xlsx"
data_att = pd.read_excel(dir_att)
df_list = data_att.values.tolist()

print("DONE HERE")

DONE HERE


In [1]:
'''
read all the files in tgt_img_dir
crop and resize each image to 128x128 using trim_to_128_by_128_bitmoji()
'''

print("STARTING")
print(df_list[0])
for i, filename in enumerate(os.scandir(orig_bitmoji_dir)):
    if filename.is_file():
        image = cv2.imread(filename.path)
        crop_img = trim_to_128_by_128_bitmoji(image) #crop image
        filename = tgt_bitmoji_dir + df_list[i][0]
        cv2.imwrite(filename, crop_img) #save image to directory
        

        
print("DONE")


STARTING
DONE
